In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 

import tqdm

In [4]:
def conv(val):
    if val == np.nan:
        return 0 # or whatever else you want to represent your NaN with
    return val

y_Ag = pd.read_csv("./dataset/Ag.csv", encoding='utf-8', index_col=0).dropna()
y_Au = pd.read_csv("./dataset/Au.csv", encoding='utf-8', index_col=0).dropna()
y_Cu = pd.read_csv("./dataset/Cu.csv", encoding='utf-8', index_col=0).dropna()
# y_Ag = pd.read_csv("./old_dataset/Ag.csv", encoding='utf-8', index_col=0).dropna()
# y_Au = pd.read_csv("./old_dataset/Au.csv", encoding='utf-8', index_col=0).dropna()
# y_Cu = pd.read_csv("./old_dataset/Cu.csv", encoding='utf-8', index_col=0).dropna()


y_dict = {"Ag":y_Ag, "Au":y_Au, "Cu":y_Cu}
indexs = np.append(y_Ag.index.to_numpy(), [y_Au.index.to_numpy(), y_Cu.index.to_numpy()])
cols = y_Ag.columns
print("indexs:", indexs)
print("cols:", cols)

indexs: ['Ag-Ti' 'Ag-V' 'Ag-Cr' 'Ag-Mn' 'Ag-Fe' 'Ag-Co' 'Ag-Ni' 'Ag-Mo' 'Ag-Zr'
 'Ag-Nb' 'Ag-Ru' 'Ag-Rh' 'Ag-Pd' 'Ag-Hf' 'Ag-Ta' 'Ag-W' 'Ag-Re' 'Ag-Os'
 'Ag-Ir' 'Ag-Pt' 'Au-Ti' 'Au-V' 'Au-Cr' 'Au-Mn' 'Au-Fe' 'Au-Co' 'Au-Ni'
 'Au-Mo' 'Au-Zr' 'Au-Nb' 'Au-Ru' 'Au-Rh' 'Au-Pd' 'Au-Hf' 'Au-Ta' 'Au-W'
 'Au-Re' 'Au-Os' 'Au-Ir' 'Au-Pt' 'Cu-Ti' 'Cu-V' 'Cu-Cr' 'Cu-Mn' 'Cu-Fe'
 'Cu-Co' 'Cu-Ni' 'Cu-Mo' 'Cu-Zr' 'Cu-Nb' 'Cu-Ru' 'Cu-Rh' 'Cu-Pd' 'Cu-Hf'
 'Cu-Ta' 'Cu-W' 'Cu-Re' 'Cu-Os' 'Cu-Ir' 'Cu-Pt']
cols: Index(['CO', 'CO2', 'H', 'H2', 'H2O', 'N', 'N2', 'O', 'OH', 'OOH', 'C', 'COH',
       'O2', 'CH3', 'CHO', 'NH3'],
      dtype='object')


In [7]:
import os
# assign directory
dir_name = "./dataset/data_spd/"
dir_list = ['Ag', 'Au', 'Cu']

import cv2

y = []
x = []
z = []
reshape_size = (2, 750)
 
# iterate over files in
# that directory
for index in tqdm.tqdm(indexs):
    metal = index.split("-")[0]
    sub_metal = index.split("-")[1]
    for col in cols:
        sub_dir = "./dataset/data_spd/%s/%s/%s/dos"%(metal, index, col) 
#         sub_dir = "./old_dataset/%s/%s/%s/dos"%(metal, index, col)
        tmp = []
        for file in os.listdir(sub_dir):
            f = os.path.join(sub_dir, file)
            # checking if it is a file
            if os.path.isfile(f):
                split = f.split('.')
                if len(split)==2: continue
                if split[2] == "dat":
                    split = f.split('-')
#                     if split[2] == "p" or split[2] == "d" or split[2] == "s" or "TDOS-aft.dat" in f:
                    dat = pd.read_csv(f,  sep="\s+", names=["DOS", "Energy"], header=None)
                    tmp.append(cv2.resize(dat.to_numpy(), dsize=reshape_size, interpolation=cv2.INTER_CUBIC)[:, 0])
#                     else:
#                         continue
        if np.array(tmp).shape[0] == 0:
            continue
        x.append(np.array(tmp))
        y.append(y_dict[metal].loc[index][col])
        z.append([metal, sub_metal, col])
y = np.array(y) # y shape (780,)
x = np.array(x)
z = np.array(z)

print("y shape", y.shape)
print("x shape", x.shape)
print("z shape", z.shape)

  0%|                                                                                                        | 0/60 [00:00<?, ?it/s]

y shape (1,)
x shape (1, 41, 526)
z shape (1, 3)


In [8]:
x = np.transpose(x, (0, 2, 1)) # shape (2640, 750, 41)
print(x.shape)

(1, 526, 41)


In [8]:
np.save("input", x)
np.save("target", y)
np.save("name", z)